In [17]:
import urllib.request
import pandas as pd

# Load the TSV file
tsv_file_path = './update_tle.tsv'

try:
    sheet_values = pd.read_csv(tsv_file_path, sep='\t')
except FileNotFoundError:
    sheet_values = pd.DataFrame(columns=['tle_0', 'tle_1', 'norad_cat_num'])

# Convert to dictionary for quick lookup
tle_dict = {row['norad_cat_num']: row for _, row in sheet_values.iterrows()}

# Fetch satellite catalog
satcat_url = "https://celestrak.com/pub/satcat.txt" # https://api.optimize.esa.int/data/debris_clouds/satcat.txt
try:
    sat_catalogue = urllib.request.urlopen(satcat_url).read().decode('utf-8').strip().split("\n")
except Exception as e:
    raise ConnectionError(f"Failed to fetch satellite catalog: {e}")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue


In [18]:
import os
from dotenv import load_dotenv
from spacetrack import SpaceTrackClient

load_dotenv('.env')  # Provide the path to your .env file

username = os.getenv('SPACETRACK_USERNAME')
password = os.getenv('SPACETRACK_PASSWORD')

st = SpaceTrackClient(username, password)
print(st)


SpaceTrackClient<identity='hello@giacomonanni.info'>


In [19]:
import time
new_entries = []

for key in sat_dict.keys():
    if str('n_' + key) not in tle_dict:
        try:
            if "-" in sat_dict[key][75:85] and "N/A" not in sat_dict[key][119:127]:
                lines = st.tle_latest(norad_cat_id=[key], format='tle')
                lines = lines.split("\n")

                for line1, line2 in zip(*[iter(lines[-3:-1])]*2):
                    update_line = [str(line1), str(line2), str('n_' + key)]
                    print(update_line)
                    new_entries.append(update_line)

                    # Limit the API request
                    time.sleep(2) # 20 advised
            else:
                continue
        except Exception as e:
            print(f"Error processing satellite {key}: {e}")


['1  9572U 75112C   23281.09328950  .08088259  19070-5  31259-2 0  9995', '2  9572  73.9823 310.2340 0008125 285.2222  74.8130 16.23822304554894', 'n_09572']
['1 09648U 76126E   23008.10893388  .12457807  69309-5  20306-2 0  9991', '2 09648  65.7240 255.5537 0011820 194.9130 165.1714 16.32922117338063', 'n_09648']
['1  9674U 72058HQ  23259.83113424  .04146798  27865-5  15622-2 0  9995', '2  9674  97.9462  15.0471 0010172 283.7626  76.2531 16.24226238755364', 'n_09674']
['1 09818U 76126AE  24022.78264937  .21412484  69268-5  35667-2 0  9995', '2 09818  65.6823  12.7416 0003020  44.0119 316.1311 16.32948262356602', 'n_09818']
['1 10059U 77047A   23004.20266218  .99999999  77959-5  23415+0 0  9994', '2 10059  61.8026  26.0040 0299563  91.1623  37.6505 15.61133018335048', 'n_10059']
['1 10229U 77065BG  24078.72674527  .01571565  00000-0  21425-1 0  9995', '2 10229  28.7726 276.3757 0020419  60.2036 300.0600 15.53434269570135', 'n_10229']
['1 10230U 77065BH  23289.57497541  .05374008  11105

In [15]:
if new_entries:
    new_df = pd.DataFrame(new_entries, columns=['tle_0', 'tle_1', 'norad_cat_num'])
    updated_sheet = pd.concat([sheet_values, new_df])
    updated_sheet.to_csv(tsv_file_path, sep='\t', index=False)
else:
    print("No new entries to update.")